In [21]:
import pandas as pd

In [22]:
# Cesty k súborom
file_control    = r"C:\Users\marti\Diplomovka\DiplomovkaOpravene.xlsx"
file_stanza     = r"C:\Users\marti\Diplomovka\stanza\outputStanza.xlsx"
file_trankit    = r"C:\Users\marti\Diplomovka\Trankit\Trankit_words.xlsx"
file_treetagger = r"C:\Users\marti\Diplomovka\TreeTagger\outputTreeTagger.xlsx"
file_udpipe     = r"C:\Users\marti\Diplomovka\UDPipe\UDPipe.xlsx"
file_morphodita = r"C:\Users\marti\Diplomovka\morphodita\words.xlsx"

In [23]:
# ------------------------------
# 1. Vytvorenie agregovaného DataFrame s UPOS tagmi
# ------------------------------

# Načítanie kontrolného datasetu a extrakcia stĺpca Xpos1 – premenovaného na "Control"
control_df = pd.read_excel(file_control)
# Predpokladáme, že kontrolný súbor obsahuje stĺpec "Xpos1"
aggregatedUPOS = pd.DataFrame()
aggregatedUPOS['Control'] = control_df['Xpos1']

# Nastavíme počet riadkov ako dĺžku kontrolného datasetu
n = len(aggregatedUPOS)

# Funkcia, ktorá načíta súbor a vráti stĺpec Xpos1, orezaný na n riadkov
def load_upos(file_path, col_name="Xpos1", n_rows=None):
    df = pd.read_excel(file_path)
    series = df[col_name]
    if n_rows is not None:
        series = series.iloc[:n_rows].reset_index(drop=True)
    return series.reset_index(drop=True)

# Načítanie UPOS tagov z ďalších súborov
aggregatedUPOS['stanza1']     = load_upos(file_stanza, col_name="Xpos1", n_rows=n)
aggregatedUPOS['trankit1']    = load_upos(file_trankit, col_name="XPOS1", n_rows=n)
aggregatedUPOS['treetagger1'] = load_upos(file_treetagger, col_name="Xpos1", n_rows=n)
aggregatedUPOS['udpipe1']     = load_upos(file_udpipe, col_name="Xpos1", n_rows=n)

# Pre súbor morphodita – načítame stĺpec "Tag" a vezmeme iba jeho prvé písmeno.
df_morphodita = pd.read_excel(file_morphodita)
# Uistíme sa, že všetko je string a vezmeme len prvý znak
morphodita_series = df_morphodita['Tag'].astype(str).str[0]
# Orezanie na n riadkov, ak je potrebné
morphodita_series = morphodita_series.iloc[:n].reset_index(drop=True)
aggregatedUPOS['morphodita1'] = morphodita_series

# Uloženie agregovaného DataFrame do Excelu
aggregatedUPOS.to_excel(r"C:\Users\marti\Diplomovka\agregovanyUPOS.xlsx", index=False)

# ------------------------------
# 2. Vytvorenie DataFrame s výsledkami porovnania UPOS tagov s kontrolou
# ------------------------------

# Vytvoríme nový DataFrame, kde bude prvý stĺpec "Control" a zvyšné budú porovnania (True/False)
match_df = pd.DataFrame()
match_df['Control'] = aggregatedUPOS['Control']

# Zoznam zdrojových stĺpcov, ktoré chceme porovnávať
columns_to_compare = ['stanza1', 'trankit1', 'treetagger1', 'udpipe1', 'morphodita1']

for col in columns_to_compare:
    # Porovnáme hodnotu v danej bunke so stĺpcom Control – výsledok je True, ak sa zhodujú, inak False
    match_df[col] = aggregatedUPOS[col] == aggregatedUPOS['Control']

# Uloženie výsledného porovnávacieho DataFrame do Excelu (voliteľne)
match_df.to_excel(r"C:\Users\marti\Diplomovka\agregovanyUPOS_Match.xlsx", index=False)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\marti\\Diplomovka\\agregovanyUPOS.xlsx'

In [ ]:
from collections import Counter
Counter(match_df['trankit1'])

Counter({False: 860, True: 339})

In [ ]:
aggregatedUPOS

,Control,stanza1,trankit1,treetagger1,udpipe1,morphodita1
0,V,V,V,V,V,V
1,T,T,T,T,T,X
2,T,D,T,T,T,D
3,S,S,S,S,S,N
4,O,O,O,O,O,J
...,...,...,...,...,...,...
1194,S,S,E,S,S,V
1195,V,V,S,V,V,A
1196,E,E,V,E,E,N
1197,S,S,A,A,S,N


In [ ]:
# Predpokladáme, že match_df obsahuje stĺpce:
# "Control", "stanza1", "trankit1", "treetagger1", "udpipe1", "morphodita1"
# a že v týchto stĺpcoch (okrem Control) sú booleovské hodnoty True/False.

total_rows = len(match_df)
success_summary = {}

# Iterujeme cez všetky stĺpce okrem "Control"
for col in match_df.columns:
    if col == "Control":
        continue
    # Počet True (match) v danom stĺpci
    count_true = match_df[col].sum()  # True sa počíta ako 1
    percentage = (count_true / total_rows) * 100
    success_summary[col] = percentage

# Výpis výsledkov
print("Percentuálny podiel úspešnosti pre jednotlivé knižnice:")
for lib, pct in success_summary.items():
    print(f"{lib}: {pct:.2f}%")


Percentuálny podiel úspešnosti pre jednotlivé knižnice:
stanza1: 96.75%
trankit1: 28.27%
treetagger1: 29.19%
udpipe1: 97.66%
morphodita1: 12.68%
